In [36]:
!pip install langgraph
!pip install langchain-google-genai



Python(53832) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Python(53843) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [37]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict, Literal, Annotated
from langchain_core.messages  import SystemMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI


In [38]:
generator_llm = ChatGoogleGenerativeAI(model="gemini-pro")
evaluator_llm= ChatGoogleGenerativeAI(model="gemini-pro")
optimizer_llm= ChatGoogleGenerativeAI(model="gemini-pro")

/Users/comida/ai-learning/.venv/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
Python(53958) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
/Users/comida/ai-learning/.venv/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
Python(539

In [39]:
from pydantic import BaseModel, Field

class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "need_improvment"]= Field(..., description="Final evalutaion")
    feedback: str =Field(...,description="feedback of the tweet")

In [40]:
structured_evaluator_llm =evaluator_llm.with_structured_output(TweetEvaluation)

In [41]:
#state
class TweetState(TypedDict):
    topic :str
    tweet: str
    evaluation: Literal["approved","needs_improvement"]
    feedback: str
    iteration: int
    max_iteration: int

In [42]:
def generate_tweet(state: TweetState):
    #prompt 
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")
    ]

    #send to generator llm 
    response=generator_llm.invoke(messages).content

    #return state
    return { "tweet": response}

def evaluate_tweet(state: TweetState):
    #prompt
    messages=[
       SystemMessage(
    content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets harshly."
),
HumanMessage(
    content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality Is this fresh, or have you seen it a hundred times before?
2. Humor  Did it genuinely make you smile, laugh, or chuckle?
3. Punchiness  Is it short, sharp, and scroll-stopping?
4. Virality Potential  Would people retweet or share it?
5. Format Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Don’t end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces are overrated.”)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"
- feedback: One paragraph explaining the strengths and weaknesses
"""
)


    ]

    #send to generator llm
    response =structured_evaluator_llm.invoke(messages)

    return {'evaluation': response.evaluation, "feedback": response.feedback}

    


def optimized_tweet(state: TweetState):
    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]

    response= optimizer_llm.invoke(messages).content
    iteration= state['iteration']+1

    return {'tweet': response, 'iteration': iteration }


In [43]:
def route_evaluation(state: TweetState):
    if state['evaluation']=='approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    else:
        return 'needs_improvement' 

In [44]:
graph =StateGraph(TweetState)

graph.add_node("generate", generate_tweet)
graph.add_node("evaluate", evaluate_tweet)
graph.add_node("optimize", optimized_tweet)


graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')
graph.add_conditional_edges('evaluate',route_evaluation, {'evaluate': END, 'needs_improvement': 'optimize'})
graph.add_edge('optimize', 'evaluate')

workflow=graph.compile()

workflow

In [45]:
intial_state={
    "topic": "indian railway",
    "iteration": 1,
    "max_iterations": 5
}

workflow.invoke(intial_state)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised PermissionDenied: 403 Request had insufficient authentication scopes. [reason: "ACCESS_TOKEN_SCOPE_INSUFFICIENT"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "method"
  value: "google.ai.generativelanguage.v1beta.GenerativeService.GenerateContent"
}
].


PermissionDenied: 403 Request had insufficient authentication scopes. [reason: "ACCESS_TOKEN_SCOPE_INSUFFICIENT"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
metadata {
  key: "method"
  value: "google.ai.generativelanguage.v1beta.GenerativeService.GenerateContent"
}
]